## NLP Class Final Project Read Data

In [1]:
import pandas as pd

In [2]:
# import modules & set up logging
import os
import pandas as pd
from pandarallel import pandarallel

import re
import nltk
import gensim

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_colwidth', 200)

In [3]:
%%time

df_news_final_project = pd.read_parquet('https://storage.googleapis.com/msca-bdp-data-open/news_final_project/news_final_project.parquet', engine='pyarrow')
df_news_final_project.shape

CPU times: user 12.9 s, sys: 16.4 s, total: 29.3 s
Wall time: 5min 4s


(199838, 5)

In [4]:
import multiprocessing

num_processors = multiprocessing.cpu_count()
print(f'Available CPUs: {num_processors}')

Available CPUs: 8


In [5]:
pandarallel.initialize(nb_workers=num_processors-1, use_memory_fs=False,)

INFO: Pandarallel will run on 7 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
pd.set_option("display.max_columns", 10)

In [7]:
df_news_final_project

,url,date,language,title,text
0,http://auckland.scoop.co.nz/2020/01/aut-boosts-ai-expertise-with-new-ailab/,2020-01-28,en,auckland.scoop.co.nz » AUT boosts AI expertise with new AiLab,\n\nauckland.scoop.co.nz » AUT boosts AI expertise with new AiLab\nScoop Search\nContact\nNewsagent\nLogin\n \n \nSupercity\nBusiness\nEducation\nEntertainment\nHealth\nPolice\nPolitics\n\n\n \n\n...
1,http://en.people.cn/n3/2021/0318/c90000-9830122.html,2021-03-18,en,Artificial intelligence improves parking efficiency in Chinese cities - People's Daily Online,\n\nArtificial intelligence improves parking efficiency in Chinese cities - People's Daily Online\n\nHome\nChina Politics\nForeign Affairs\nOpinions\nVideo: We Are China\nBusiness\nMilitary\nWorld...
2,http://galusaustralis.com/2020/02/486473/legaltech-artificial-intelligence-market-2019-technology-advancement-and-future-scope-casetext-inc-catalyst-repository-systems-ebrevia/,2020-02-26,en,"LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis","LegalTech Artificial Intelligence Market 2019 Technology Advancement and Future Scope – Casetext Inc., Catalyst Repository Systems, eBREVIA – Galus Australis \n\nGalus Australis\n\nBusine..."
3,http://newsparliament.com/2020/02/27/children-with-autism-saw-their-learning-and-social-skills-boosted-after-playing-with-this-ai-robot/,2020-02-27,en,Children With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament,"\nChildren With Autism Saw Their Learning and Social Skills Boosted After Playing With This AI Robot – News Parliament\n \n\nSkip to content\n\t\t\tThursday, February 27, 2020\t\t\n\nLatest:\n\n\n..."
4,http://spaceref.com/astronomy/observation-simulation-and-ai-join-forces-to-reveal-a-clear-universe.html,2021-07-05,en,"Observation, Simulation, And AI Join Forces To Reveal A Clear Universe - SpaceRef","\n\nObservation, Simulation, And AI Join Forces To Reveal A Clear Universe - SpaceRef\n\n \nHome |\nNASA Watch\nSpaceRef Business\nAstrobiology Web\nAdvertising\nAdd an Event\nSign up for our Dail..."
...,...,...,...,...,...
199833,https://www.yourcentralvalley.com/news/tech-news/un-urges-moratorium-on-use-of-ai-that-imperils-human-rights/,2021-09-15,en,UN urges moratorium on use of AI that imperils human rights | YourCentralValley.com KSEE24 | CBS47,"\n\n \nUN urges moratorium on use of AI that imperils human rights | YourCentralValley.com KSEE24 | CBS47\n\nSkip to content\n\n\nYourCentralValley.com\nFresno, CA\n\n70°\nSponsored By\n\n \nTogg..."
199834,https://www.zdnet.com/article/csiro-using-artificial-intelligence-to-map-1-7m-australian-grain-paddocks/,2020-08-07,en,CSIRO using artificial intelligence to map 1.7m Australian grain paddocks | ZDNet,\n\nCSIRO using artificial intelligence to map 1.7m Australian grain paddocks | ZDNet\n\n \n\n Edition: \n\n Asia\n \n Australia\n ...
199835,https://yemenwatch.com/escaping-the-singularity-why-artificial-intelligence-will-not-save-the-planet-18/,2021-01-15,en,Escaping the Singularity: Why Artificial Intelligence Will Not Save the Planet | Yemen Watch,\n\nEscaping the Singularity: Why Artificial Intelligence Will Not Save the Planet | Yemen Watch\nHome\nNews\nBusiness\nMagazine\nSports\nCulture\nPolitics\n \nSign in\nWelcome!Log into your acco...
199836,https://yourstory.com/2022/12/google-ai-model-indian-languages-sundar-pichai-ashwini-vaishnaw,2022-12-19,en,Google building AI model to support over 100 Indian languages: Sundar Pichai,Google building AI model to support over 100 Indian languages: Sundar PichaiMenuGo to HomeStories & MediaBrands of New IndiaSMB StoryYS GulfHerStorySocialStoryEnterprise StoryThe Decrypting StoryT...


In [6]:
news_df = df_news_final_project

In [7]:
%%time
news_df["clean_text"] = news_df["text"].parallel_apply(lambda x: re.sub(r'\n', '',x)) # remove new line
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'https://.*', '',x)) # remove url
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'@\w*', '',x)) # remove mention
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'#\w*', '',x)) # remove tag
# news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'[^a-zA-Z.\s]', '',x)) # keep only letters, periods, and white space
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(' +', ' ',x)) # change consecutive white space into 1 whitespace

CPU times: user 11.9 s, sys: 56.7 s, total: 1min 8s
Wall time: 8min 3s


In [8]:
%%time
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'\xa0SAP', '',x))
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'\xa0National', '',x))
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'\xa0', '',x))
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'\t', '',x))
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'\r', '',x))
news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'\|', '',x))

CPU times: user 13.1 s, sys: 60 s, total: 1min 13s
Wall time: 7min 1s


In [12]:
news_df.text[4]

'\n\nObservation, Simulation, And AI Join Forces To Reveal A Clear Universe - SpaceRef\n\n \nHome |\nNASA Watch\nSpaceRef Business\nAstrobiology Web\nAdvertising\nAdd an Event\nSign up for our Daily Newsletter \n\n \n\n\nInternational Space Station\nNASA Hack Space\nCalendar\nMissions\nSpace Weather \n             \xa0             \n           \nObservation, Simulation, And AI Join Forces To Reveal A Clear Universe\n\n\n             Press Release - Source: NATIONAL INSTITUTES OF NATURAL SCIENCES\n             \n            \nPosted July  4, 2021 10:00 PM\n View Comments\nUsing AI driven data analysis to peel back the noise and find the actual shape of the Universe. CREDIT The Institute of Statistical Mathematics\nJapanese astronomers have developed a new artificial intelligence (AI) technique to remove noise in astronomical data due to random variations in galaxy shapes.\nAfter extensive training and testing on large mock data created by supercomputer simulations, they then applied thi

In [23]:
news_df.clean_text[4]

'Observation, Simulation, And AI Join Forces To Reveal A Clear Universe - SpaceRef Home NASA WatchSpaceRef BusinessAstrobiology WebAdvertisingAdd an EventSign up for our Daily Newsletter International Space StationNASA Hack SpaceCalendarMissionsSpace Weather  Observation, Simulation, And AI Join Forces To Reveal A Clear Universe Press Release - Source: NATIONAL INSTITUTES OF NATURAL SCIENCES Posted July 4, 2021 10:00 PM View CommentsUsing AI driven data analysis to peel back the noise and find the actual shape of the Universe. CREDIT The Institute of Statistical MathematicsJapanese astronomers have developed a new artificial intelligence (AI) technique to remove noise in astronomical data due to random variations in galaxy shapes.After extensive training and testing on large mock data created by supercomputer simulations, they then applied this new tool to actual data from Japan\'s Subaru Telescope and found that the mass distribution derived from using this method is consistent with t

In [9]:
%%time
news_df["clean_title"] = news_df["title"].parallel_apply(lambda x: re.sub(r'\n', '',x)) # remove new line
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'https://.*', '',x)) # remove url
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'@\w*', '',x)) # remove mention
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'#\w*', '',x)) # remove tag
# news_df["clean_text"] = news_df["clean_text"].parallel_apply(lambda x: re.sub(r'[^a-zA-Z.\s]', '',x)) # keep only letters, periods, and white space
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(' +', ' ',x)) # change consecutive white space into 1 whitespace

CPU times: user 377 ms, sys: 682 ms, total: 1.06 s
Wall time: 2.28 s


In [10]:
%%time
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'\xa0SAP', '',x))
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'\xa0National', '',x))
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'\xa0', '',x))
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'\t', '',x))
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'\r', '',x))
news_df["clean_title"] = news_df["clean_title"].parallel_apply(lambda x: re.sub(r'\|', '',x))

CPU times: user 442 ms, sys: 473 ms, total: 915 ms
Wall time: 1.17 s


In [34]:
%%time

top_N = 100

clean_text = news_df['clean_text']

clean_text = str(clean_text)


# Use TweetTokenizer
tweet_tokenizer = nltk.tokenize.TweetTokenizer()
words = tweet_tokenizer.tokenize(clean_text)

#stopwords = stopwords.words('english')
stopwords = set(nltk.corpus.stopwords.words('english'))

# Remove single-character tokens (mostly punctuation)
words = [word for word in words if len(word) > 1]

# Remove numbers
words = [word for word in words if not word.isnumeric()]

# Remove punctuation
# words = [word for word in words if word.isalpha()]

# Lowercase all words (default_stopwords are lowercase too)
# words = [word.lower() for word in words]

# Remove stopwords
words = [word for word in words if word not in stopwords]

bgs = nltk.bigrams(words)

#compute frequency distribution for all the bigrams in the text
fdist_2 = nltk.FreqDist(bgs)

fdist_2_df = pd.DataFrame(fdist_2.most_common(),
                    columns=['Word', 'Frequency'])

# fdist_2_df.head(n=20)

CPU times: user 8.35 ms, sys: 289 µs, total: 8.64 ms
Wall time: 6.68 ms


In [30]:
fdist_2_df.head(n=21)

,Word,Frequency
0,"(Artificial, Intelligence)",3
1,"(boosts, AI)",2
2,"(AI, expertise)",2
3,"(expertise, new)",2
4,"(..., Artificial)",2
5,"(auckland.scoop.co.nz, AUT)",1
6,"(AUT, boosts)",1
7,"(new, AiLabScoop)",1
8,"(AiLabScoop, SearchContactNewsagentLogin)",1
9,"(SearchContactNewsagentLogin, SupercityBusinessEducationEntertainmentHealthPolicePolitics)",1


In [31]:
%%time

top_N = 100

clean_title = news_df['clean_title']

clean_title = str(clean_title)


# Use TweetTokenizer
tweet_tokenizer = nltk.tokenize.TweetTokenizer()
words = tweet_tokenizer.tokenize(clean_title)

#stopwords = stopwords.words('english')
stopwords = set(nltk.corpus.stopwords.words('english'))

# Remove single-character tokens (mostly punctuation)
words = [word for word in words if len(word) > 1]

# Remove numbers
words = [word for word in words if not word.isnumeric()]

# Remove punctuation
# words = [word for word in words if word.isalpha()]

# Lowercase all words (default_stopwords are lowercase too)
# words = [word.lower() for word in words]

# Remove stopwords
words = [word for word in words if word not in stopwords]

bgs = nltk.bigrams(words)

#compute frequency distribution for all the bigrams in the text
fdist_2 = nltk.FreqDist(bgs)

fdist_2_df = pd.DataFrame(fdist_2.most_common(),
                    columns=['Word', 'Frequency'])

# fdist_2_df.head(n=20)

CPU times: user 1.3 ms, sys: 6.07 ms, total: 7.37 ms
Wall time: 5.88 ms


In [32]:
fdist_2_df.head(n=20)

,Word,Frequency
0,"(Artificial, Intelligence)",3
1,"(auckland.scoop.co.nz, AUT)",1
2,"(AUT, boosts)",1
3,"(boosts, AI)",1
4,"(AI, expertise)",1
5,"(expertise, new)",1
6,"(new, AiLab)",1
7,"(AiLab, Artificial)",1
8,"(Artificial, intelligence)",1
9,"(intelligence, improves)",1


In [33]:
fdist = nltk.FreqDist(words)
fdist.most_common(10)

[('AI', 5),
 ('Artificial', 4),
 ('Intelligence', 3),
 ('intelligence', 2),
 ('With', 2),
 ('Learning', 2),
 ('auckland.scoop.co.nz', 1),
 ('AUT', 1),
 ('boosts', 1),
 ('expertise', 1)]

In [36]:
fdist = nltk.FreqDist(words)
fdist.most_common(20)

[('...', 9),
 ('AI', 6),
 ('Artificial', 4),
 ('ZDNet', 4),
 ('Intelligence', 3),
 ('With', 3),
 ('boosts', 2),
 ('expertise', 2),
 ('new', 2),
 ('intelligence', 2),
 ('Daily', 2),
 ('Galus', 2),
 ('Learning', 2),
 ('United', 2),
 ('auckland.scoop.co.nz', 1),
 ('AUT', 1),
 ('AiLabScoop', 1),
 ('SearchContactNewsagentLogin', 1),
 ('SupercityBusinessEducationEntertainmentHealthPolicePolitics', 1),
 ('TweetAUT', 1)]

In [37]:
min_date = news_df['date'].min()
max_date = news_df['date'].max()

print('Data ranges from {} to {}'.format(min_date, max_date))

Data ranges from 2020-01-01 to 2023-02-07


In [11]:
keywords = ['Data Science', 'data science', 'DATA SCIENCE','AI', 'ai', 'artificial intelligence','Artificial Intelligence','ARTIFICIAL INTELLIGENCE', 'ML', 'NLP', 
            'Artificial General Intelligence','Chatbots', 'AI Marketplaces', 'Intelligent Applications', 
           'Augmented Intelligence', 'Decision Intelligence', 'AI Cloud Services', 'cloud services', 'GPU Accelerators', 'Computer Vision','Deep Neural Network',
           'Deep Learning','Cognitive Computing','Autonomous Vehicles','Knowledge Graphs', 'Responsible AI', 'Machine Customers', 'Decision Intelligence', 'Autonomous Vehicles',
           'Human-Centered AI', 'AI Governance', 'Natural Language Processing', 'Machine Learning', 'Smart Robots', 'Operational AI Systems' , 'Data-Centric AI', 'AI TRiSM', 'Generative AI',
           'Responsible AI']
query = '|'.join(keywords)
news_df = news_df[news_df['clean_text'].str.contains(query)]
news_df = news_df[news_df['clean_title'].str.contains(query)]
news_df.shape

(186913, 7)

In [12]:
news_df = news_df[["date","clean_title","clean_text"]].drop_duplicates()

In [13]:
news_df.to_parquet("nlp_clean_text.parquet", index = False)